<a href="https://colab.research.google.com/github/yonseimath/datascience-biginner-2022-kaggle-competitions/blob/feature%2Fjuwon/juwon/220708_10_%EC%9E%90%EC%97%B0%EC%96%B4_%EA%B3%B5%EB%B6%80(XGBoost).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI4_Code_XGBoost

In [ ]:
import json
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit #
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer #
from xgboost import XGBRanker
from bisect import bisect

sklearn.model_selection

배열 또는 행렬을 무작위 학습 및 테스트 하위 집합으로 분할합니다.

입력 유효성 검사 및 입력 데이터에 대한 응용 프로그램을 oneliner에서 데이터 분할(및 선택적으로 서브샘플링)을 위한 단일 호출로 래핑하는 빠른 유틸리티입니다


shuffle: bool, default=True

분할하기 전에 데이터를 섞을지 여부입니다. shuffle=False이면 stratify는 None이어야 합니다.

sklearn.feature_extraction.text

텍스트 문서 모음을 토큰 수의 행렬로 변환합니다.
사전 사전을 제공하지 않고 일종의 기능 선택을 수행하는 분석기를 사용하지 않는 경우 기능의 수는 데이터를 분석하여 찾은 어휘 크기와 동일합니다.


sklearn.feature_extraction.text.TfidfVectorizer

원시 문서 모음을 TF-IDF 기능의 매트릭스로 변환합니다.



In [ ]:
num_train = 10000 # 10000개를 돌릴 것이다.?

def read_notebook(path,id_name): # 노트북 읽기
    return (
        pd.read_json(path, dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=id_name)     #Object.assign() 메서드는 출처 객체들의 모든 열거 가능한 자체 속성을 복사해 대상 객체에 붙여넣습니다. 그 후 대상 객체를 반환합니다.

        .rename_axis('cell_id') # DataFrame.rename_axis(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False)
                                # 인덱스 또는 열의 축 이름을 설정합니다.
                                # axis{0 or ‘index’, 1 or ‘columns’}, default 0<< 이름을 바꿀 축입니다.
    )

paths = []
directory = '../input/AI4Code/train'
for file in os.scandir(directory):
    if file.is_file():
        paths.append(file.path) # 파일이 파일이면 path에 추가됨. 
    if len(paths) == num_train: # 이 둘이 다르면 끝낸다. 원하는 수를 앞에 정해뒀으니. 
        break
        
id_names = []
for name in paths:
    name = name.split('/')
    id_n = name[-1].split('.')
    id_names.append(id_n[0])
    
# print(id_names)
# print(paths)

train_notebooks = []
for i in range(len(paths)): # path 길이의 수. 하나하나 넣기.
    train_notebooks.append(read_notebook(paths[i],id_names[i]))
    
print(train_notebooks[0])



이걸 실행 하면 cell_id 랑 cell_type 이랑 source를 정리해줌. 우리가 처음에 공부했던 거랑 같은 방식의 셋팅 

In [ ]:
# print(type(train_notebooks[0]))
df = (
    pd.concat(train_notebooks) # 문자열 텍스트 결합 
    .set_index('id', append=True) # 인덱스를 id로 하고 계속 추가가 가능..?
    .swaplevel() # swaplevel메서드는 Multi Index (또는 Munti Columns)에서 두 인덱스의 순서를 변경하는 메서드입니다. 왜 하는 지는 잘은 몰겟지만 한번 빼고 돌려보면 차이를 알지 않을까?
                # 순서를 변경해야 보기가 훨씬 편합니다. 돌려봤음.
                # DataFrame.swaplevel(i=- 2, j=- 1, axis=0); i , j : 순서를 변경할 두 인덱스의 레벨입니다. 기본적으로 제일 낮은 두 레벨의 인덱스가 교환됩니다.(-2, -1)
                # axis : 기본값은 0
    .sort_index(level='id', sort_remaining=False) # 인덱스 정렬 
)
df.head() # head(self, n=5) DataFrame 내의 처음 n줄의 데이터를 출력한다. 이 메서드(method)는 객체 안에 제대로된 데이터 타입이 입력되어있는지 빠르게 확인할 경우 사용하면 매우 유용하다. 디폴트가 5.


In [ ]:
df_orders = pd.read_csv('../input/AI4Code/train_orders.csv',index_col='id',squeeze=True).str.split() # csv 파일 읽어오기. np.squeeze() 함수 : 크기가 1인 axis 제거(이건 일단 보류)
                                                                                                                        # 데이터 압축을 위해 
df_orders

In [ ]:
df_orders_ = df_orders.to_frame().join( #아까 order 만든 걸 frame 으로 만드는 것 같은데 조인을 하네.
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list), # id로 그룹 짓고 리스트 형식이고..? 인 'cell_id'
    how='right', # right outer join
)

def get_ranks(base, derived): #순위 할당. AI4Code Pytorch DistilBert Baseline이랑 XGBoost이랑 비슷한 방식인가? 
    return [base.index(d) for d in derived] # d가 저장되어있는 위치를 반환합니다.

ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)} # 여기서 d의 정체가 나온다. 

df_ranks = (
    pd.DataFrame
    .from_dict(ranks, orient='index') # 배열과 같은 dict 또는 dict에서 DataFrame을 구성한다. orient='columns', dtype: 강제로 적용할 데이터 유형입니다. 없으면 유추합니다.
                                      # 열 또는 dtype 지정을 허용하는 인덱스로 사전 DataFrame 객체 생성.
                                      # Column labels to use when orient='index'. Raises a ValueError if used with orient='columns' or orient='tight'.
                                      # 데이터의 "방향". 전달된 딕셔너리의 키가 있는 경우, 결과 DataFrame의 열이어야 하며 '열'(기본값)을 전달합니다.                       
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)

df_ranks

In [ ]:
df_ancestors = pd.read_csv('../input/AI4Code/train_ancestors.csv', index_col='id')
df_ancestors

In [ ]:
siz = 0.1  # size of validation set, 검증 세트의 크기

splitter = GroupShuffleSplit(n_splits=1, test_size=siz, random_state=0) # n_splits 는 데이터 분할 수 입니다. Shuffle-Group(s)-Out 교차 검증 반복기
                                                                        # 제3자 제공 그룹에 따라 데이터를 분할하기 위한 무작위 학습/테스트 인덱스를 제공합니다. 
                                                                        # 이 그룹 정보는 샘플의 임의의 도메인 특정 계층을 정수로 인코딩하는 데 사용할 수 있습니다.
                                                                        # test_size float인 경우 0.0에서 1.0 사이여야 하며 테스트 분할(반올림)에 포함할 그룹의 비율을 나타냅니다. 
                                                                        #int이면 테스트 그룹의 절대 수를 나타냅니다.
                                                                        # random_state : 생성된 훈련 및 테스트 인덱스의 무작위성을 제어합니다. 여러 함수 호출에서 재현 가능한 출력을 위해 int를 전달합니다. 
# Split, keeping notebooks with a common origin (ancestor_id) together
ids = df.index.unique('id')                                             # 이부분은 https://www.kaggle.com/code/choione/getting-started-with-ai4code-study 여기 랑 방식이 같다. 
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

In [ ]:
# Training set. 여기도 https://www.kaggle.com/code/choione/getting-started-with-ai4code-study 여기 랑 방식이 같다.
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str)) # X 는 train에서 quality 를 제외한 모든 변수
                                                              # y 는 train의 qulity 변수
# Rank of each cell within the notebook
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()


In [ ]:
# Add code cell ordering
# 이제 코드 셀 순서를 기능으로 추가해 보겠습니다.
# 올바른 순서로 코드 셀을 열거하는 열을 추가합니다.like 1, 2, 3, 4, ..., 모든 마크다운 셀에 대해 더미 값 0을 갖습니다.
# 이 기능은 모델이 코드 셀을 올바른 순서로 배치하는 방법을 배우는 데 도움이 됩니다.

X_train = sparse.hstack(( # 희소 행렬(행렬의 값이 대부분 0인 경우를 가리키는 표현)을 수평으로 쌓기(열별)
    X_train,
    np.where(
        df_train['cell_type'] == 'code', # 셀타입이 코드인걸로 
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1, # 220703에 공부한 내용 
        0,
    ).reshape(-1, 1)
))
print(X_train.shape)

In [ ]:
# from xgboost import XGBRanker
model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)
model.fit(X_train, y_train, group=groups)

Kendall tau 상관 관계의 변형을 사용하여 검증 세트에 대한 예측을 평가합니다.

먼저 훈련 세트와 마찬가지로 검증 세트에 대한 기능을 생성합니다.

In [ ]:
# Validation set
X_valid = tfidf.transform(df_valid['source'].astype(str))
# The metric uses cell ids
y_valid = df_orders.loc[ids_valid]

X_valid = sparse.hstack((
    X_valid,
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

여기에서 모델을 사용하여 노트북 내 각 셀의 순위를 예측한 다음 이 순위를 정렬된 셀 ID 목록으로 변환합니다.

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index) # predict 메소드와 test_one 데이터를 이용해 품질 예측

y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)
y_pred.head(10)

이제 노트북을 살펴보고 모델이 어떻게 되었는지 살펴보겠습니다.

In [ ]:
nb_id = df_valid.index.get_level_values('id').unique()[8]

display(df.loc[nb_id])
display(df.loc[nb_id].loc[y_pred.loc[nb_id]])


In [ ]:
def count_inversions_slowly(ranks): # 계산을 더 빠르게 하는 역할 같음. ...?
    inversions = 0
    size = len(ranks)
    for i in range(size):
        for j in range(i+1, size):
            if ranks[i] > ranks[j]:
                total += 1
    return total

def count_inversions(a): # 배열의 반전 횟수 는 배열이 정렬되는 데 얼마나 멀리(또는 가까운지) 나타냅니다. 
                        # 배열이 이미 정렬되어 있으면 반전 횟수는 0이지만 배열을 역순으로 정렬하면 반전 횟수가 최대입니다. 
                        #공식적으로 말하면, a[i] > a[j] 및 i < j인 경우 두 요소 a[i] 및 a[j]가 반전을 형성합니다.
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):  # enumerate() 함수는 인자로 넘어온 목록을 기준으로 인덱스와 원소를 차례대로 접근하게 해주는 반복자(iterator) 객체를 반환해주는 함수입니다. 
        j = bisect(sorted_so_far, u)  # 이 모듈은 정렬된 리스트를 삽입 후에 다시 정렬할 필요 없도록 관리할 수 있도록 지원합니다.
        inversions += i - j
        sorted_so_far.insert(j, u)  
    return inversions

def kendall_tau(ground_truth, predictions): # 측정된 두 양 사이 의 순서 관계 를 측정하는 데 사용되는 통계 
                                            # 통계적 의존성에 대한 비모수 가설 검정
    total_inversions = 0  
    total_2max = 0  
    for gt, pred in zip(ground_truth, predictions): # zip() 여러 개의 순회 가능한(iterable) 객체를 인자로 받고, 
                                                    # 각 객체가 담고 있는 원소를 터플의 형태로 차례로 접근할 수 있는 반복자(iterator)를 반환합니다.
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth. 예측 순서 매기기.
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

섞인 노트북의 ID에서 생성된 더미 제출로 메트릭을 테스트해 보겠습니다.

In [ ]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy) 

이것을 예측의 점수와 비교하면 우리 모델이 실제로 셀 순서를 어느 정도 향상시킬 수 있음을 알 수 있습니다.

In [ ]:
kendall_tau(y_valid, y_pred)

In [ ]:
paths = []
directory = '../input/AI4Code/test'
for file in os.scandir(directory):
    if file.is_file():
        paths.append(file.path)
    if len(paths) == num_train:
        break
        
id_names = []
for name in paths:
    name = name.split('/')
    id_n = name[-1].split('.')
    id_names.append(id_n[0])
    
# print(id_names)
# print(paths)

test_notebooks = []
for i in range(len(paths)):
    test_notebooks.append(read_notebook(paths[i],id_names[i]))
    
print(test_notebooks[0])

In [ ]:
# print(type(train_notebooks[0]))
df_test = (
    pd.concat(test_notebooks)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
df_test.head()

In [ ]:
X_test = tfidf.transform(df_test['source'].astype(str))
X_test = sparse.hstack((
    X_test,
    np.where(
        df_test['cell_type'] == 'code',
        df_test.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

In [ ]:
y_infer = pd.DataFrame({'rank': model.predict(X_test)}, index=df_test.index)
y_infer = y_infer.sort_values(['id', 'rank']).reset_index('cell_id').groupby('id')['cell_id'].apply(list)
y_infer

In [ ]:
y_submit = (
    y_infer
    .apply(' '.join)  # list of ids -> string of ids
    .rename_axis('id')
    .rename('cell_order')
)
y_submit

In [ ]:
y_submit.to_csv('submission.csv') # y_submit.to_csv('submission.csv', index=False) 로 써야 할까? 
                                  #결과가 안 나오는 이유는 뭐지?
                                  #y_submit.to_csv('submission.csv', index = 'False')를 해봤지만 변하는 건 없었다.. 